In [1]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
# from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import csr_matrix
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
RawTrajs = pd.read_csv('../RawLidarData/Veteran/OutputFile/OutputTrajs/Trajctories.csv')
print(RawTrajs.columns)
TrajGroups = RawTrajs.groupby('ObjectID')
Trajs = [traj[1].loc[:,['Coord_X_est','Coord_Y_est','Coord_Lon_est','Coord_Lat_est','Speed_x','Speed_y']].reset_index(drop = True) for traj in TrajGroups]
Lens = [len(traj) for traj in Trajs]

In [27]:
Label,Attribute = [],[]
for ind,traj in tqdm(enumerate(Trajs)):
    if Lens[ind] > 2:
        for i in range(1,len(traj)):
            labels = traj.iloc[i,[0,1,4,5]]
            attributes = traj.iloc[i-1,[0,1,4,5]]
            Label.append(labels)
            Attribute.append(attributes)
    else:
        continue 

2658it [00:55, 47.61it/s]


In [28]:
Label = pd.concat(Label,axis = 1).T
Attribute = pd.concat(Attribute,axis = 1).T

In [56]:
Label.to_csv('../PosteriorData/Label.csv',index = False)
Attribute.to_csv('../PosteriorData/Attribute.csv',index = False)

In [2]:
os.getcwd()

'e:\\Lidar-Tracker\\Lidar-Tracker'